In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)

In [3]:
print(word_dict)

{'coffee': 0, 'dog': 1, 'like': 2, 'love': 3, 'milk': 4, 'hate': 5, 'i': 6}


In [4]:
print(n_class)

7


In [33]:
input_batch, target_batch = [], []
for sen in sentences:
    word = sen.split()
    print(word)
    input1 = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]
    print(input1)
    
    input_batch.append(np.eye(n_class)[input1])
    target_batch.append(target)

['i', 'like', 'dog']
[6, 2]
['i', 'love', 'coffee']
[6, 3]
['i', 'hate', 'milk']
[6, 5]


In [34]:
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [35]:
print(input_batch)
print(input_batch.shape)

tensor([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0.]]])
torch.Size([3, 2, 7])


In [31]:
target_batch

tensor([1, 0, 4])

In [36]:
input_batch1 = input_batch.transpose(0,1)
print(input_batch1)
print(input_batch1.shape)
# 이러면 안되는거 아닌가? 
# 문장이 3개인데 2개로 분류되면 3개의 문장이 2개로 섞이는데...


# 잘못생각했네 rnn의 특성상 sequence한 다음 데이터를 예측하기 위해서 앞단어와 다음 단어를 분리한거네 

tensor([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.]]])
torch.Size([2, 3, 7])


In [37]:
rnn = nn.RNN(input_size = n_class, hidden_size = n_hidden)
W = nn.Linear(n_hidden, n_class, bias = False)
b = nn.Parameter(torch.ones([n_class]))


In [44]:
outputs,hidden = rnn(input_batch1)

In [45]:
print(outputs)
print(outputs.shape)

tensor([[[ 0.2912, -0.5669,  0.1427,  0.3485,  0.4932],
         [ 0.2912, -0.5669,  0.1427,  0.3485,  0.4932],
         [ 0.2912, -0.5669,  0.1427,  0.3485,  0.4932]],

        [[ 0.1172, -0.6765,  0.0636,  0.2234,  0.7838],
         [ 0.2039, -0.6064, -0.1160,  0.2507,  0.5864],
         [ 0.4487, -0.6771,  0.5561, -0.2510,  0.3231]]],
       grad_fn=<StackBackward>)
torch.Size([2, 3, 5])


In [46]:
outputs1 = outputs[-1]
print(outputs1)

tensor([[ 0.1172, -0.6765,  0.0636,  0.2234,  0.7838],
        [ 0.2039, -0.6064, -0.1160,  0.2507,  0.5864],
        [ 0.4487, -0.6771,  0.5561, -0.2510,  0.3231]],
       grad_fn=<SelectBackward>)


In [47]:
model = W(outputs1) + b
print(model)

tensor([[1.3234, 1.1556, 0.4484, 1.0180, 0.8028, 0.9104, 1.0893],
        [1.3022, 1.1605, 0.6169, 1.0219, 0.9380, 0.8566, 0.9660],
        [1.0822, 0.9184, 0.5364, 0.8814, 1.0252, 1.3371, 1.1767]],
       grad_fn=<AddBackward0>)


In [49]:
criterion = nn.CrossEntropyLoss()
loss = criterion(model, target_batch)
print(loss)

tensor(1.7906, grad_fn=<NllLossBackward>)
